In [1]:
from helper import *

2025-01-15 14:04:00.207313: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-15 14:04:00.215486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736949840.224665    1483 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736949840.227826    1483 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-15 14:04:00.238353: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU available")

1 Physical GPUs, 1 Logical GPUs


I0000 00:00:1736949861.675217    1483 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13499 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4080 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:
data = pd.read_csv("data.csv")
print(data.shape)

(5572, 7480)


In [5]:
X = data["text_clean"]
X = pd.Series(map(str, X))
y = data["y"]
print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5572,) (5572,)
(4457,) (4457,)
(1115,) (1115,)


In [6]:
MAX_NUM_WORDS = 2000
MAX_SEQUENCE_LENGTH = 100
X_sequence_train , tok = tokenize_text(X_train, MAX_NUM_WORDS, MAX_SEQUENCE_LENGTH)
X_sequence_test = pad_sequences(tok.texts_to_sequences(X_test), maxlen=MAX_SEQUENCE_LENGTH)
print(X_sequence_train.shape, X_sequence_test.shape)

(4457, 100) (1115, 100)


In [4]:
def make_model(name, 
               embedding_dim=128, 
               lstm_units=128, 
               dense_units=64,
               dropout_rate=0.2,
            ):
    model = Sequential(name=name)
    model.add(Embedding(input_dim=MAX_NUM_WORDS, output_dim=embedding_dim))
    model.add(LSTM(units=lstm_units, dropout=dropout_rate, recurrent_dropout=dropout_rate))
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.build(input_shape=(None, MAX_SEQUENCE_LENGTH))
    model.summary()
    return model
        
param_grid = {
        'embedding_dim': [50, 100],
        'lstm_units': [24,32,48],
        'dense_units': [32,64,96],
        'dropout_rate': [0.1,0.2, 0.4],
    }   


In [7]:
r = grid_search(param_grid, make_model, X_sequence_train , y_train , X_sequence_test , y_test)


Grid size: 54


Model: "Model0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 50)        │       100,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 48)             │        19,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 96)             │         4,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            97 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 123,809 (483.63 KB)

 Trainable params: 123,809 (483.63 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_53 (Embedding)        │ (None, 100, 100)       │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_53 (LSTM)                  │ (None, 48)             │        28,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_106 (Dense)               │ (None, 96)             │         4,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_107 (Dense)               │ (None, 1)              │            97 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 233,409 (911.75 KB)

 Trainable params: 233,409 (911.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 536ms/step - accuracy: 0.8391 - loss: 0.4917 - val_accuracy: 0.9305 - val_loss: 0.2562
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 17s 538ms/step - accuracy: 0.9491 - loss: 0.1947 - val_accuracy: 0.9574 - val_loss: 0.1480
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 18s 568ms/step - accuracy: 0.9843 - loss: 0.0777 - val_accuracy: 0.9641 - val_loss: 0.1126
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 583ms/step - accuracy: 0.9895 - loss: 0.0400 - val_accuracy: 0.9664 - val_loss: 0.0986
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 621ms/step - accuracy: 0.9933 - loss: 0.0229 - val_accuracy: 0.9641 - val_loss: 0.1116
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 618ms/step - accuracy: 0.9958 - loss: 0.0152 - val_accuracy: 0.9664 - val_loss: 0.1162
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 610ms/step - accuracy: 0.9973 - loss: 0.0139 - val_accuracy: 0.9664 - val_loss: 0.1351
35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 139ms/step
Model: Model53
Parameters: {'embedding_dim': 100, 'lstm_

In [8]:
from pprint import pprint

r.sort(key=lambda x: x[2]["classification_report"]["accuracy"], reverse=True)
pprint([ (params , metrics["classification_report"]["accuracy"]) for model, params, metrics in r])

[({'dense_units': 96,
   'dropout_rate': 0.4,
   'embedding_dim': 100,
   'lstm_units': 48,
   'name': 'Model53'},
  0.9874439461883409),
 ({'dense_units': 96,
   'dropout_rate': 0.4,
   'embedding_dim': 50,
   'lstm_units': 48,
   'name': 'Model52'},
  0.9874439461883409),
 ({'dense_units': 96,
   'dropout_rate': 0.4,
   'embedding_dim': 50,
   'lstm_units': 48,
   'name': 'Model52'},
  0.9874439461883409),
 ({'dense_units': 96,
   'dropout_rate': 0.4,
   'embedding_dim': 50,
   'lstm_units': 48,
   'name': 'Model52'},
  0.9865470852017937),
 ({'dense_units': 96,
   'dropout_rate': 0.4,
   'embedding_dim': 100,
   'lstm_units': 48,
   'name': 'Model53'},
  0.9865470852017937),
 ({'dense_units': 96,
   'dropout_rate': 0.4,
   'embedding_dim': 100,
   'lstm_units': 48,
   'name': 'Model53'},
  0.9865470852017937),
 ({'dense_units': 96,
   'dropout_rate': 0.4,
   'embedding_dim': 50,
   'lstm_units': 48,
   'name': 'Model52'},
  0.9865470852017937),
 ({'dense_units': 96,
   'dropout_rate

In [12]:
# save models in directory models
for model, params, metrics in r:
    model.save(f"models/{model.name}.keras")


In [7]:
X2 = data["text"]
X2 = pd.Series(map(str, X2))
y = data["y"]
print(X2.shape, y.shape)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, test_size=TEST_SPLIT, random_state=42)
print(X2_train.shape, y2_train.shape)
print(X2_test.shape, y2_test.shape)

(5572,) (5572,)
(4457,) (4457,)
(1115,) (1115,)


In [9]:
MAX_NUM_WORDS = 2000
MAX_SEQUENCE_LENGTH = 100
X2_sequence_train , tok2 = tokenize_text(X2_train, MAX_NUM_WORDS, MAX_SEQUENCE_LENGTH)
X2_sequence_test = pad_sequences(tok2.texts_to_sequences(X2_test), maxlen=MAX_SEQUENCE_LENGTH)
print(X2_sequence_train.shape, X2_sequence_test.shape)

(4457, 100) (1115, 100)


In [11]:
r2 = grid_search(param_grid, make_model, X2_sequence_train , y2_train , X2_sequence_test , y2_test)

Grid size: 54


Model: "Model0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 50)        │       100,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 24)             │         7,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 108,033 (422.00 KB)

 Trainable params: 108,033 (422.00 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_53 (Embedding)        │ (None, 100, 100)       │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_53 (LSTM)                  │ (None, 48)             │        28,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_106 (Dense)               │ (None, 96)             │         4,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_107 (Dense)               │ (None, 1)              │            97 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 233,409 (911.75 KB)

 Trainable params: 233,409 (911.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 569ms/step - accuracy: 0.8199 - loss: 0.5178 - val_accuracy: 0.8587 - val_loss: 0.2499
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 18s 565ms/step - accuracy: 0.9254 - loss: 0.1873 - val_accuracy: 0.9641 - val_loss: 0.1262
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 18s 580ms/step - accuracy: 0.9845 - loss: 0.0630 - val_accuracy: 0.9709 - val_loss: 0.0867
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 604ms/step - accuracy: 0.9911 - loss: 0.0364 - val_accuracy: 0.9731 - val_loss: 0.0958
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 635ms/step - accuracy: 0.9937 - loss: 0.0273 - val_accuracy: 0.9709 - val_loss: 0.0810
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 631ms/step - accuracy: 0.9972 - loss: 0.0113 - val_accuracy: 0.9686 - val_loss: 0.1061
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 630ms/step - accuracy: 0.9957 - loss: 0.0142 - val_accuracy: 0.9686 - val_loss: 0.1212
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 603ms/step - accuracy: 0.9984 - loss: 0.0091 - val_accu

In [12]:
from pprint import pprint

r2.sort(key=lambda x: x[2]["classification_report"]["accuracy"], reverse=True)
pprint([ (params , metrics["classification_report"]["accuracy"]) for model, params, metrics in r2])

[({'dense_units': 64,
   'dropout_rate': 0.1,
   'embedding_dim': 50,
   'lstm_units': 32,
   'name': 'Model8'},
  0.9847533632286996),
 ({'dense_units': 96,
   'dropout_rate': 0.1,
   'embedding_dim': 50,
   'lstm_units': 32,
   'name': 'Model14'},
  0.9847533632286996),
 ({'dense_units': 32,
   'dropout_rate': 0.1,
   'embedding_dim': 100,
   'lstm_units': 24,
   'name': 'Model1'},
  0.9838565022421525),
 ({'dense_units': 32,
   'dropout_rate': 0.1,
   'embedding_dim': 50,
   'lstm_units': 32,
   'name': 'Model2'},
  0.9838565022421525),
 ({'dense_units': 32,
   'dropout_rate': 0.1,
   'embedding_dim': 50,
   'lstm_units': 48,
   'name': 'Model4'},
  0.9838565022421525),
 ({'dense_units': 32,
   'dropout_rate': 0.1,
   'embedding_dim': 100,
   'lstm_units': 48,
   'name': 'Model5'},
  0.9838565022421525),
 ({'dense_units': 64,
   'dropout_rate': 0.1,
   'embedding_dim': 100,
   'lstm_units': 24,
   'name': 'Model7'},
  0.9838565022421525),
 ({'dense_units': 64,
   'dropout_rate': 0.1

In [13]:
for model, params, metrics in r2:
    model.save(f"models2/{model.name}.keras")
